In [5]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
from PIL import Image, ImageDraw , ImageFont
import folium
%matplotlib inline
from folium import plugins
import os
import io
os.environ["PATH"] += os.pathsep + 'phantomjs'

/Users/souvikroy/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
taxi_data=pd.read_csv('train.csv')

In [ ]:
taxi_data.to_csv('trainingDataWithUniqueID.csv')

In [ ]:
taxi_data=pd.read_csv('trainingDataWithUniqueID.csv')

In [ ]:
taxi_data.head(3)

In [ ]:
list(taxi_data)

In [ ]:
def calculate_pickup_hour(x):
    pickup_time=datetime.strptime(x['pickup_datetime'], '%Y-%m-%d %H:%M:%S')
    return pickup_time.hour

def calculate_dropoff_hour(x):
    dropoff_time=datetime.strptime(x['dropoff_datetime'], '%Y-%m-%d %H:%M:%S')
    return dropoff_time.hour


In [ ]:
taxi_data['pickup_hour']=taxi_data.apply(calculate_pickup_hour,axis=1)
taxi_data['dropoff_hour']=taxi_data.apply(calculate_dropoff_hour,axis=1)

In [ ]:
import folium
from folium import plugins
 

In [ ]:
taxi_heatmap = folium.Map(location=[taxi_data_reduced['pickup_latitude'].mean(), taxi_data_reduced['pickup_longitude'].mean()], zoom_start=10)
taxi_heatmap.add_children(plugins.HeatMap([[row['pickup_latitude'], row['pickup_longitude']] for name, row in taxi_data_reduced.iterrows()]))
taxi_heatmap.save("heatmap_pickup.html")
taxi_heatmap

In [ ]:
taxi_heatmap = folium.Map(location=[taxi_data_reduced['dropoff_latitude'].mean(), taxi_data_reduced['dropoff_longitude'].mean()], zoom_start=10)
taxi_heatmap.add_children(plugins.HeatMap([[row['dropoff_latitude'], row['dropoff_longitude']] for name, row in taxi_data_reduced.iterrows()]))
taxi_heatmap.save("heatmap_dropoff.html")
taxi_heatmap

In [ ]:
from IPython.display import HTML

def display(m, height=300):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

In [ ]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")
marker = folium.CircleMarker(location=[40.738, -73.98])
marker.add_to(folium_map)

In [ ]:
taxi_data_with_pickup_info=pd.read_csv('taxi_data_pickup_lat_long_with_info.csv')

In [ ]:
taxi_data=taxi_data.rename(index=str, columns={"UniqueID": "uniqueid"})

In [ ]:
taxi_data_full=pd.merge(taxi_data_with_pickup_info, taxi_data, how='inner', on='uniqueid')

In [ ]:
taxi_data_full=taxi_data_full.rename(index=str, columns={"county": "pickup_county", "city": "pickup_city","name":"pickup_neighborhood"})

In [ ]:
list(taxi_data_full)

In [ ]:
taxi_data_full=taxi_data_full[['uniqueid', 'pickup_county', 'pickup_city', 'pickup_neighborhood', 'pickup_datetime', 'dropoff_datetime', 'passenger_count','pickup_latitude', 'pickup_longitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'pickup_hour', 'dropoff_hour','id','vendor_id']]

In [ ]:
taxi_data_full[['uniqueid','dropoff_latitude', 'dropoff_longitude']].to_csv('taxi_data_dropoff_lat_long.csv',index=False)

In [ ]:
taxi_data_dropoff_with_info=pd.read_csv('taxi_data_dropoff_lat_long_with_info.csv')

In [ ]:
taxi_data_with_all_info=pd.merge(taxi_data_full,taxi_data_dropoff_with_info,how='inner',on='uniqueid')

In [ ]:
taxi_data_with_all_info=taxi_data_with_all_info.rename(index=str, columns={"county": "dropoff_county", "city": "dropoff_city","name":"dropoff_neighborhood"})

In [ ]:
taxi_data_with_all_info=taxi_data_with_all_info[['uniqueid', 'pickup_county', 'pickup_city', 'pickup_neighborhood', 'pickup_datetime', 'dropoff_datetime', 'passenger_count','pickup_latitude', 'pickup_longitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'pickup_hour', 'dropoff_hour','pickup_date','dropoff_city','dropoff_county','dropoff_neighborhood','vendor_id','id']]

In [ ]:
taxi_data_with_all_info['pickup_date']=taxi_data_with_all_info['pickup_datetime'].apply(lambda x: x.split()[0])

In [ ]:
weather=pd.read_csv('WeatherData.csv')

In [ ]:
weather_refined=weather[['PRCP','SNOW','SNWD','TMAX','TMIN','DATE']]

In [ ]:
weather_refined=weather_refined.rename(index=str,columns={'DATE':'pickup_date'})


In [ ]:
taxi_data_with_all_info=pd.merge(weather_refined,taxi_data_with_all_info,how='inner',on='pickup_date')

In [ ]:
taxi_data_with_all_info.head(3)

In [ ]:
taxi_data_with_all_info.to_csv('taxi_data_with_all_info.csv',index=False)

In [ ]:
trip_time_by_county=taxi_data_with_all_info.groupby(['pickup_county','dropoff_county'],as_index=False).agg({'trip_duration':'mean'})


In [ ]:
trip_time_by_neighborhood=taxi_data_with_all_info.groupby(['pickup_neighborhood','dropoff_neighborhood'],as_index=False).agg({'trip_duration':'mean'})


In [ ]:
trip_time_by_city=taxi_data_with_all_info.groupby(['pickup_city','dropoff_city'],as_index=False).agg({'trip_duration':'mean'})


In [ ]:
taxi_data_with_all_info=taxi_data_with_all_info.rename(index=str, columns={"trip_duration": "trip_duration_by_county"})

In [ ]:
trip_time_by_county=trip_time_by_county.rename(index=str, columns={"trip_duration": "trip_duration_by_county"})

In [ ]:
trip_time_by_neighborhood=trip_time_by_neighborhood.rename(index=str, columns={"trip_duration": "trip_duration_by_neighborhood"})

In [ ]:
trip_time_by_city=trip_time_by_city.rename(index=str, columns={"trip_duration": "trip_duration_by_city"})

In [ ]:
taxi_data_with_all_info=pd.merge(taxi_data_with_all_info,trip_time_by_county,on=['pickup_county','dropoff_county'])

In [ ]:
taxi_data_with_all_info=pd.merge(taxi_data_with_all_info,trip_time_by_neighborhood,on=['pickup_neighborhood','dropoff_neighborhood'])

In [ ]:
taxi_data_with_all_info=pd.merge(taxi_data_with_all_info,trip_time_by_city,on=['pickup_city','dropoff_city'])

In [ ]:
list(taxi_data_with_all_info)

In [ ]:
training_features=['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'passenger_count', 'pickup_hour', 'trip_duration_by_county', 'trip_duration','trip_duration_by_neighborhood', 'trip_duration_by_city']

In [ ]:
taxi_data_final_set=taxi_data_with_all_info[training_features]

In [ ]:
list(taxi_data_with_all_info)

In [6]:
taxi_data_with_all_info=pd.read_csv('taxi_data_with_all_info.csv')

In [7]:
taxi_data_with_all_info=taxi_data_with_all_info.rename(index=str, columns={"pickup_neighboorhood": "pickup_neighborhood"})

In [8]:
taxi_data_with_all_info['pickup_neighborhood'] = taxi_data_with_all_info['pickup_neighborhood'].astype('category')
taxi_data_with_all_info['pickup_neighborhood_id']=taxi_data_with_all_info['pickup_neighborhood'].cat.codes

In [9]:
taxi_data_with_all_info['dropoff_neighborhood'] = taxi_data_with_all_info['dropoff_neighborhood'].astype('category')
taxi_data_with_all_info['dropoff_neighborhood_id']=taxi_data_with_all_info['dropoff_neighborhood'].cat.codes

In [10]:
def get_trip_counts_by_hour_pickup(selected_hour):

    locations = taxi_data_with_all_info.groupby("pickup_neighborhood_id").first()
    locations = locations[["pickup_latitude","pickup_longitude","pickup_neighborhood"]]
    
    #select pickup time
    subset = taxi_data_with_all_info[taxi_data_with_all_info["pickup_hour"]==selected_hour]
    
    # count trips for each pickup neighborhood
    departure_counts =  subset.groupby("pickup_neighborhood_id").count()
    departure_counts = departure_counts.iloc[:,[0]]
    departure_counts.columns= ["Departure Count"]
    
    trip_counts = departure_counts.join(locations)
    return trip_counts

# print a sample
get_trip_counts_by_hour_pickup(3).head()

,Departure Count,pickup_latitude,pickup_longitude,pickup_neighborhood
pickup_neighborhood_id,,,,
1,246,40.774471,-73.913033,Astoria
2,1,40.768295,-73.899490,Astoria Heights
4,1,40.599155,-73.989265,Bath Beach
5,28,40.711201,-74.016022,Battery Park
6,2,40.624229,-74.030792,Bay Ridge


In [17]:
def plot_counts_pickup(trip_counts,day):
    # generate a new map
    loc=[40.738, -73.98]
    zs=12
    w='80%'
    
    if day:
        
        folium_map = folium.Map(location=loc,
                            zoom_start=zs,
                            tiles="cartodbpositron",
                            width=w)
    else:
        
        folium_map = folium.Map(location=loc,
                            zoom_start=zs,
                            tiles="CartoDB dark_matter",
                            width=w)

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        #departures
        net_departures = (row["Departure Count"])
   
        # radius of circles
        radius = net_departures/250 
            
        # add marker to the map
        folium.CircleMarker(location=(row["pickup_latitude"],
                                      row["pickup_longitude"]),
                            radius=radius,
                            color="#ff0000",
                            fill=True).add_to(folium_map)
        
    return folium_map

In [18]:
def interpolate(df1, df2, x):
    """return a weighted average of two dataframes"""
    df = df1 * (1 - x) + df2 * x
    return df.replace(np.nan, 0)
  

def get_trip_counts_by_minute_pickup(float_hour, data):
    """get an interpolated dataframe for any time, based
    on hourly data"""
    
    columns = ["pickup_latitude",
               "pickup_longitude",
               "Departure Count"]
               
    df1 = get_trip_counts_by_hour_pickup(int(float_hour))
    df2 = get_trip_counts_by_hour_pickup(int(float_hour) + 1)
    
    df = interpolate(df1.loc[:,columns], 
                     df2.loc[:,columns], 
                     float_hour % 1)
    
    df["pickup_neighborhood"] = df1["pickup_neighborhood"]
    return df

In [19]:
data = get_trip_counts_by_minute_pickup(9.5, taxi_data_with_all_info)
plot_counts_pickup(data,False)

In [20]:
def frame_pickup(i, hour_of_day, save_path):
    
    # create the map object
    data = get_trip_counts_by_minute_pickup(hour_of_day, taxi_data_with_all_info)
    my_frame = plot_counts_pickup(data,hour_of_day<17)
    
    # generate the png file as a byte array
    png = my_frame._to_png()
    
    #  now add a caption to the image to indicate the time-of-day.
    hour = int(hour_of_day)
    minutes = int((hour_of_day % 1) * 60)
    
    # create a PIL image object
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)
    
    # load a font
    font = ImageFont.truetype("Montserrat-Regular.ttf",30)
    
    # draw time of day text
    draw.text((20,image.height - 50), 
              "time: {:0>2}:{:0>2}h".format(hour, minutes),
              fill=(255, 0, 0), 
              font=font)
    
    # draw title
    draw.text((image.width - 400,20), 
              "Pickups vs Time of Day",
              fill=(255, 0, 0), 
              font=font)
    
    # write to a png file
    filename = os.path.join(save_path, "frame_{:0>5}.png".format(i))
    image.save(filename, "PNG")
    return image

In [21]:
def get_trip_counts_by_hour_dropoff(selected_hour):
    # make a DataFrame with locations for each bike station
    locations = taxi_data_with_all_info.groupby("dropoff_neighborhood_id").first()
    locations = locations[["dropoff_latitude","dropoff_longitude","dropoff_neighborhood"]]
    
    #select one time of day
    subset = taxi_data_with_all_info[taxi_data_with_all_info["dropoff_hour"]==selected_hour]
    
    # count trips for each destination
    arrival_counts =  subset.groupby("dropoff_neighborhood_id").count()
    arrival_counts = arrival_counts.iloc[:,[0]]
    arrival_counts.columns= ["Arrival Count"]
    

    #join departure counts, arrival counts, and locations
    trip_counts = arrival_counts.join(locations)
    return trip_counts

# print a sample to check our code works
get_trip_counts_by_hour_dropoff(6).head()

,Arrival Count,dropoff_latitude,dropoff_longitude,dropoff_neighborhood
dropoff_neighborhood_id,,,,
4,2,40.587467,-73.802635,Arverne
5,284,40.765919,-73.918854,Astoria
6,2,40.767582,-73.889427,Astoria Heights
7,3,40.761570,-73.792847,Auburndale
8,3,40.599152,-73.989220,Bath Beach


In [22]:
def plot_counts_dropoff(trip_counts,day):
    # generate a new map
    
    #loc=[40.7245, -73.9419]
    loc=[40.738, -73.98]
    zs=13
    w='100%'
    
    if day:
        
        folium_map = folium.Map(location=loc,
                            zoom_start=zs,
                            tiles="cartodbpositron",
                            width=w)
    else:
        
        folium_map = folium.Map(location=loc,
                            zoom_start=zs,
                            tiles="CartoDB dark_matter",
                            width=w)

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        
        net_arrivals = (row["Arrival Count"])

        
        # radius of circles
        radius = net_arrivals/250
        
       
        # add marker to the map
        folium.CircleMarker(location=(row["dropoff_latitude"],
                                      row["dropoff_longitude"]),
                            radius=radius,
                            color="#00FF00",
                            fill=True).add_to(folium_map)
    

    return folium_map

In [23]:
def get_trip_counts_by_minute_dropoff(float_hour, data):
    """get an interpolated dataframe for any time, based
    on hourly data"""
    
    columns = ["dropoff_latitude",
               "dropoff_longitude",
               "Arrival Count",]
               
    df1 = get_trip_counts_by_hour_dropoff(int(float_hour))
    df2 = get_trip_counts_by_hour_dropoff(int(float_hour) + 1)
    
    df = interpolate(df1.loc[:,columns], 
                     df2.loc[:,columns], 
                     float_hour % 1)
    
    df["dropoff_neighborhood"] = df1["dropoff_neighborhood"]
    return df

In [24]:
data = get_trip_counts_by_minute_dropoff(9.5, taxi_data_with_all_info)
plot_counts_dropoff(data,False)

In [25]:
def frame_dropoff(i, hour_of_day, save_path):
    
    # create the map object
    data = get_trip_counts_by_minute_dropoff(hour_of_day, taxi_data_with_all_info)
    my_frame = plot_counts_dropoff(data,hour_of_day<17)
    
    # generate the png file as a byte array
    png = my_frame._to_png()
    
    #  now add a caption to the image to indicate the time-of-day.
    hour = int(hour_of_day)
    minutes = int((hour_of_day % 1) * 60)
    
    # create a PIL image object
    image = Image.open(io.BytesIO(png))
    draw = ImageDraw.ImageDraw(image)
    
    # load a font
    font = ImageFont.truetype("Montserrat-Regular.ttf",30)
    
    # draw time of day text
    draw.text((20,image.height - 50), 
              "time: {:0>2}:{:0>2}h".format(hour, minutes),
              fill=(0,255,0), 
              font=font)
    
    # draw title
    draw.text((image.width - 400,20), 
              "Drop-offs vs Time of Day",
              fill=(0,255,0), 
              font=font)
    
    # write to a png file
    filename = os.path.join(save_path, "frame_{:0>5}.png".format(i))
    image.save(filename, "PNG")
    return image

In [26]:
times  = np.arange(5, 24, 0.5)
for i,hour in enumerate(times):
        frame_pickup(i, hour, "frames_pickup_Manhattan")
        frame_dropoff(i, hour, "frames_dropoff_Manhattan")

/Users/souvikroy/anaconda2/lib/python2.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


KeyboardInterrupt: 

In [ ]:
map_new = folium.Map(location=[40.7245, -73.9419],
                    zoom_start = 11.5) 

# List comprehension to make out list of lists
heat_data = [[[row['pickup_latitude'],row['pickup_longitude']] for index, row in taxi_data_with_all_info[taxi_data_with_all_info['pickup_hour'] == i].iterrows()] for i in range(5,24)]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.5)
hm.add_to(map_new)

In [ ]:
map_new.save("map_pickup.html")